# Particle physics results

## Setup

In [1]:
import sys
import numpy as np
from sklearn.metrics import roc_auc_score


## Load results

In [2]:
show_all = True
n_runs = 3
n_chains = 5
remove_all_results_with_nans = False

In [3]:
algo_filenames = []
algo_additionals = []
algo_labels = []
algo_dividers = []

def add_algo(filename, add, label, shortlist=False):
    if show_all or shortlist:
        algo_filenames.append(filename)
        algo_additionals.append(add)
        algo_labels.append(label)
    
    
def add_divider():
    algo_dividers.append(len(algo_filenames))
    

add_algo("flow", "_march", "AF")
add_algo("pie", "_march", "PIE")
add_algo("mf", "_alternate_march", "FLM-A")
add_algo("gamf", "_march", "FLM-OT")
add_algo("gamf", "_alternate_march", "FLM-OTA")
add_algo("emf", "_alternate_march", "FLME-A")


In [12]:
def load(name, shape, numpyfy=True, chains=1, result_dir="../data/results"):
    all_results = []
    
    for algo_filename, algo_add in zip(algo_filenames, algo_additionals):
        algo_results = []
            
        for run in range(n_runs):
            run_str = "" if run == 0 else "_run{}".format(run)
            
            for chain in range(chains):
                chain_str = "" if chain == 0 else "_chain{}".format(chain)
                
                try:
                    this_result = np.load(
                        "{}/{}_14_lhc{}{}_{}{}.npy".format(
                            result_dir, algo_filename, algo_add, run_str, name, chain_str
                        )
                    )
                    if (not numpyfy) or (shape is None) or np.product(this_result.shape) == np.product(shape):
                        algo_results.append(this_result.reshape(shape))
                    else:
                        algo_results.append(np.nan*np.ones(shape))

                except FileNotFoundError as e:
                    if shape is None:
                        algo_results.append(None)
                    else:
                        algo_results.append(np.nan*np.ones(shape))
            
        all_results.append(algo_results)
    
    if numpyfy:
        all_results = np.array(all_results, dtype=np.float)
        
        if chains > 1:
            all_results = all_results.reshape([all_results.shape[0], n_runs, n_chains] + list(shape))
        
    return all_results


model_gen_x = load("samples", None, numpyfy=False)
model_gen_closure = load("samples_manifold_distance", (10000,))
model_posterior_samples = load("posterior_samples", (400, 2,), chains=n_chains)


In [15]:
model_posterior_samples.shape

(6, 3, 5000, 2)

In [5]:
def remove_nans(*results):
    passes = all([np.all(np.isfinite(result)) for result in results])
    
    if passes:
        return results
    else:
        return [np.nan * np.ones_like(result) for result in results]


def remove_nans_from_lists(*raws):
    # raws[quantity][algo]
    n_quantities = len(raws)
    n_algos = len(raws[0])
    
    for raw in raws:
        assert len(raw) == n_algos
    
    cleans = [[[] for _ in range(n_algos)] for _ in range(n_quantities)]
    
    for i in range(n_algos):
        for k in range(n_runs):
            clean = remove_nans(*[raw[i][k] for raw in raws])
            for j in range(n_quantities):
                cleans[j][i].append(clean[j])
            
    cleans = [np.array(clean) for clean in cleans]
    
    # cleans[quantity][algo]
    return cleans
     
    
if remove_all_results_with_nans:
    model_gen_closure, model_posterior_samples = remove_nans_from_lists(model_gen_closure, model_posterior_samples)


## Calculate metrics

In [6]:
n_observed = 100
min_logp = -100.

model_gen_mean_closure = np.mean(model_gen_closure, axis=2)
model_posterior_mean_distance = np.mean(np.sum(model_posterior_samples**2, axis=-1)**0.5, axis=2)


## Best metrics

In [7]:
best_closure, best_posterior_distance = -1, -1

best_closure = np.nanargmin(np.nanmean(model_gen_mean_closure, axis=1))
print(algo_labels[best_closure])

best_posterior_distance = np.nanargmin(np.nanmean(model_posterior_mean_distance, axis=1))
print(algo_labels[best_posterior_distance])


PIE
EF


/Users/johannbrehmer/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: Mean of empty slice
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/johannbrehmer/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: Mean of empty slice
  


## Print metrics

In [8]:
def print_results(
    include_err=False, include_n_runs=False, include_runs=False,
    l_label=max([len(l) for l in algo_labels]), l_mean=5, l_err=3,
    median=False, latex=False, after_decs=(3,3)
):
    # How to format the numbers
    l_result = l_mean + int(include_err) * (2 + l_err) + int(include_n_runs) * 4 + int(include_runs) * (3 + 3*l_mean + 2*2)
    l_total = l_label + 1 + 2 * (3 + l_result)
        
    # Divider
    empty_result = "" if latex else " "*(l_result + 1)
    col_divider = "&" if latex else "|"
    line_end = r"\\" if latex else ""
    block_divider = r"\midrule" if latex else "-"*l_total
    
    def _f(val, after_dec, best=False):
        if not np.any(np.isfinite(val)):
            return empty_result
        
        if median:
            result = "{:>{}.{}f}".format(np.nanmedian(val), l_mean, after_dec)
        else:
            result = "{:>{}.{}f}".format(np.nanmean(val), l_mean, after_dec)
            
        if latex and best:
            result = r"\textbf{" + result + "}"
            
        if include_err:
            result += "({:0>{}d})".format(int(10**after_dec * np.nanstd(val)), l_err)
        if include_n_runs:
            result += " [{:1n}]".format(np.sum(np.isfinite(val)))
        if include_runs:
            result += " [{:>{}.{}f}, ".format(float(val[0]), l_mean, after_dec)
            result += "{:>{}.{}f}, ".format(float(val[1]), l_mean, after_dec)
            result += "{:>{}.{}f}]".format(float(val[2]), l_mean, after_dec)
            
        if (not latex) and best:
            result += "*"
        elif (not latex) and (not best):
            result += " "
            
        return result
    
    # Header
    print(
        "{2:<{0}.{0}s} {5} {3:>{1}.{1}s} {5} {4:>{1}.{1}s} {5} {6}".format(
            l_label, l_result, "", "Closure", "Posterior mean error", col_divider, line_end
        )
    )
    print(block_divider)

    # Iterate over methods
    for i, (label, closure, distance) in enumerate(zip(
        algo_labels, model_gen_mean_closure, model_posterior_mean_distance
    )):
        # Divider
        if i in algo_dividers and show_all:
            print(block_divider)
            
        # Print results
        print("{1:<{0}.{0}s} {4} {2}{4} {3}{4} {5}".format(
            l_label, label,
            _f(closure, after_decs[0], i==best_closure), _f(distance, after_decs[1], i==best_posterior_distance),
            col_divider, line_end
        ))


In [9]:
print_results(include_runs=True)

        |                     Closure |        Posterior mean error | 
--------------------------------------------------------------------
EF      | 0.065 [0.091, 0.060, 0.044] | 0.304 [  nan,   nan, 0.304]*| 
PIE     | 0.043 [0.078, 0.024, 0.026]*| 1.213 [  nan,   nan, 1.213] | 
MLF-A   | 0.078 [0.054, 0.072, 0.106] |                             | 
MLF-OT  | 0.166 [0.193,   nan, 0.140] |                             | 
MLF-OTA |                             |                             | 
EMLF-A  | 0.222 [0.220, 0.230, 0.215] |                             | 


In [10]:
print_results(include_err=True)

        |    Closure | Posterior  | 
----------------------------------
EF      | 0.065(019) | 0.304(000)*| 
PIE     | 0.043(024)*| 1.213(000) | 
MLF-A   | 0.078(021) |            | 
MLF-OT  | 0.166(026) |            | 
MLF-OTA |            |            | 
EMLF-A  | 0.222(006) |            | 


In [11]:
print_results(include_err=True, latex=True)

        &    Closure & Posterior  & \\
\midrule
EF      & 0.065(019)& \textbf{0.304}(000)& \\
PIE     & \textbf{0.043}(024)& 1.213(000)& \\
MLF-A   & 0.078(021)& & \\
MLF-OT  & 0.166(026)& & \\
MLF-OTA & & & \\
EMLF-A  & 0.222(006)& & \\
